# 初期設定

In [1]:
from typing import List
import os
from glob import glob
from datetime import datetime

import numpy as np
import pandas as pd
from nptyping import NDArray
import numba

# 機械学習ライブラリ
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score, KFold, TimeSeriesSplit
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, classification_report, f1_score, r2_score
from sklearn.linear_model import Ridge, RidgeCV, LassoCV, LogisticRegression

INPUT_DIR = os.path.join('..', 'features')

## データの読み込み

In [3]:
input_data_path = os.path.join(INPUT_DIR, 'bybit.pkl.bz2')
df_features = pd.read_pickle(input_data_path)
df_features

,timestamp,close,buy_size_1,buy_size_2,buy_price_1,buy_price_2,sell_size_1,sell_size_2,sell_price_1,sell_price_2,...,DEMA,MACD_macd,MACD_macdsignal,MACD_macdhist,TEMA10,TEMA30,trade_signal_macd,trade_signal_bb,ask_bit_spread,size_ratio
87,2021-09-05 23:57:15.001627,50250.0,5002370.0,261469.0,50249.5,50249.0,2479452.0,500.0,50250.0,50251.0,...,50233.387927,-19.876795,-23.340356,3.463561,50244.371152,50224.005137,0,0,0.5,0.495655
88,2021-09-05 23:57:20.001557,50250.0,4648618.0,204301.0,50249.5,50249.0,752913.0,1.0,50250.0,50250.5,...,50233.335639,-17.896456,-22.251576,4.355120,50248.733752,50225.633310,0,0,0.5,0.161965
89,2021-09-05 23:57:25.001585,50250.0,4227320.0,254582.0,50249.5,50249.0,2521223.0,1.0,50250.0,50250.5,...,50233.359243,-16.140960,-21.029453,4.888492,50251.510507,50227.227435,0,0,0.5,0.596412
90,2021-09-05 23:57:30.001533,50250.0,2655342.0,254558.0,50249.5,50249.0,4262308.0,255364.0,50250.0,50250.5,...,50233.449164,-14.581629,-19.739888,5.158259,50253.167602,50228.780752,0,0,0.5,1.605182
91,2021-09-05 23:57:35.001609,50249.5,2233474.0,9131.0,50249.5,50249.0,4956121.0,243707.0,50250.0,50250.5,...,50233.534311,-13.233645,-18.438639,5.204994,50253.821845,50230.197132,0,0,0.5,2.219019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15119,2021-09-20 21:00:02.000816,43731.5,16858.0,1832.0,43730.5,43730.0,3370061.0,452500.0,43731.0,43731.5,...,43743.388409,14.686828,15.280547,-0.593719,43737.899929,43740.298128,-1,0,0.5,199.908708
15120,2021-09-20 21:00:07.000853,43691.5,2254475.0,20002.0,43691.5,43691.0,634857.0,19301.0,43692.0,43694.5,...,43739.001165,10.677770,14.359992,-3.682222,43717.756757,43733.045666,0,0,0.5,0.281599
15121,2021-09-20 21:00:12.000787,43687.0,345398.0,20002.0,43661.5,43661.0,2708842.0,2599.0,43662.0,43664.5,...,43734.402125,7.056112,12.899216,-5.843104,43702.222330,43725.772650,0,0,0.5,7.842668
15122,2021-09-20 21:00:17.001051,43660.5,1032628.0,339970.0,43673.0,43672.5,1030147.0,95001.0,43673.5,43674.0,...,43726.871011,2.024259,10.724224,-8.699965,43680.131922,43714.516275,0,0,0.5,0.997597


## 使用する特徴量

In [4]:
features = [
    'size_ratio',
    'ask_bit_spread',
    'MACD_macd',
    'MACD_macdsignal',
    'MACD_macdhist',
    'BBANDS_upperband',
    'BBANDS_middleband',
    'BBANDS_lowerband',
    'SMA',
    'EMA',
    'DEMA',
    'TEMA10',
    'TEMA30'
]